In [1]:
import polars as pl
import altair as alt
import matplotlib.pyplot as plt

In [2]:
data = pl.read_csv("./datasets/results/pop_compare.csv")
data

FileNotFoundError: No such file or directory (os error 2): ./datasets/results/pop_compare.csv

In [4]:
origin = pl.read_excel("./datasets/original/poi_original.xlsx")
origin["Population"]

FileNotFoundError: no workbook found at path './datasets/original/poi_original.xlsx'

In [ ]:
data = data.with_columns(((pl.col("Official Estimate") - pl.col("Dataset Estimate"))*100/pl.col("Official Estimate")).alias("percent off"))

In [ ]:
# Create a bar chart comparing Dataset Estimate and Official Estimate
plt.figure(figsize=(10, 6))
cities = data['DISTRICT']

# Plotting the estimates
plt.bar(cities, data['Dataset Estimate'], width=0.4, label='Dataset Estimate', align='center')
plt.bar(cities, data['Official Estimate'], width=0.4, label='Official Estimate', align='edge')

# Adding labels and title
plt.xlabel('District')
plt.ylabel('Population Estimate')
plt.title('Comparison of Dataset Estimate(Last updated 18 February 2023) vs. Official Estimate(02/2021)')
plt.xticks(rotation=90)
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
data.select(pl.col("percent off")).mean()

In [5]:
full = pl.read_excel("./datasets/temp/around_poi_with_population.xlsx")
full

FileNotFoundError: no workbook found at path './datasets/temp/around_poi_with_population.xlsx'

In [ ]:
import folium


m = folium.Map(location=(21.022672, 105.831461), zoom_start=14)

poi_id = 105
poi = full.filter(pl.col("poi_id").eq(poi_id)).to_dicts()

folium.Marker(
        location=[float(poi[0]["lat"]), float(poi[0]["lon"])],
        tooltip="Click me!",
        popup="",
        icon=folium.Icon(color="green"),
    ).add_to(m)

for place in poi:
    folium.Marker(
        location=[float(place["latitude"]), float(place["longitude"])],
        tooltip=place["title"],
        icon=folium.Icon(color="red"),
    ).add_to(m)
m

In [ ]:
origin

In [7]:
full.join(origin, left_on='poi_id', right_on='Unique Identifier')["poi_id", "population", "Population"]

NameError: name 'full' is not defined

In [5]:
data

COUNTRY,CITY,DISTRICT,TYPE,Dataset Estimate,Official Estimate
str,str,str,str,f64,f64
"""Vietnam""","""Hà Nội""","""Đống Đa""","""Quận""",478.032,401.7
"""Vietnam""","""Hà Nội""","""Đông Anh""","""Huyện""",427.819,374.9
"""Vietnam""","""Hà Nội""","""Hoàng Mai""","""Quận""",424.121,364.9
"""Vietnam""","""Hà Nội""","""Hai Bà Trưng""","""Quận""",369.003,315.9
"""Vietnam""","""Hà Nội""","""Chương Mỹ""","""Huyện""",364.014,309.6
…,…,…,…,…,…
"""Vietnam""","""Hà Nội""","""Thanh Oai""","""Huyện""",196.813,185.4
"""Vietnam""","""Hà Nội""","""Hoàn Kiếm""","""Quận""",182.718,155.9
"""Vietnam""","""Hà Nội""","""Đan Phượng""","""Huyện""",181.365,154.3


In [6]:
dataset_chart = alt.Chart(data).mark_bar().encode(
    x='DISTRICT:N',
    y='Dataset Estimate:Q'
)
official_chart = alt.Chart(data).mark_tick(color='red').encode(
    x='DISTRICT:N',
    y='Official Estimate:Q',
)
dataset_chart | official_chart

alt.HConcatChart(...)

In [2]:
df = pl.read_parquet("./datasets/raw/area_pois.parquet")
df.unique(subset=["id"])

id,lat,lon,name,categories
str,f64,f64,str,str
"""ChIJL6_ReglTNDERTOHUDTn6hFk""",20.969596,105.7642,"""Milk Tea One""","""cafe,food,point_of_interest,st…"
"""ChIJk3PQQpSsNTERV7JbwK8dVYk""",20.991791,105.809811,"""Cafe Dung""","""cafe,food,point_of_interest,es…"
"""ChIJQw3-YwCpNTERobzBYHmjPdI""",21.022204,105.901342,"""Công ty TBYT""","""hospital,point_of_interest,hea…"
"""ChIJWxoqXvarNTERMXgpjb29mA0""",21.028906,105.834663,"""FIS Hàng Bột - Trường mầm non …","""school,point_of_interest,estab…"
"""ChIJjzcAiOerNTERV4RJkkdQSCE""",21.073139,105.814767,"""RideBikes.cc - Cửa Hàng Xe Đạp""","""bicycle_store,cafe,store,food,…"
…,…,…,…,…
"""ChIJ_a39XO6tNTERfxQBX8IfIIo""",20.963813,105.821319,"""Sinh coffee roaster""","""cafe,food,point_of_interest,st…"
"""ChIJt6_O8bCuNTER3gSeCvf1Phc""",20.994177,105.886273,"""Thanh Tri Secondary School""","""school,point_of_interest,estab…"
"""ChIJ82xBS3atNTERpvYWytdfDfc""",20.978669,105.835935,"""Trường Mầm Non IQ ĐỊnh Công""","""point_of_interest,school,estab…"


In [3]:
import folium


m = folium.Map(location=(21.022672, 105.831461), zoom_start=14)

places = df.unique(subset=["id"]).sample(n=100).to_dicts()

for place in places:
    folium.Marker(
        location=[float(place["lat"]), float(place["lon"])],
        tooltip=place["name"],
        icon=folium.Icon(color="red"),
    ).add_to(m)
m

In [14]:
df = pl.read_csv("./pop.csv")
df

area,source,population
str,str,f64
"""Hà Nội""","""meta""",8.2747e6
"""Hà Nội""","""ghs""",7.4322e6
"""Cao Bằng""","""meta""",513973.673537
"""Cao Bằng""","""ghs""",482588.60852
"""Đắk Lắk""","""meta""",2.0680e6
…,…,…
"""Cần Thơ""","""tctk""",1.2589e6
"""Hậu Giang""","""tctk""",728300.0
"""Sóc Trăng""","""tctk""",1.1988e6


In [36]:
# Create the visualization
chart = alt.Chart(df).mark_bar().encode(
    y=alt.Y('area', title="District"),
    x=alt.X('population', title='Population'),
    color=alt.Color('source', title="Source"),
    yOffset="source",
).properties(width=800, height=900)
chart

alt.Chart(...)

In [23]:
    df.pivot("source", index="area").select(
        (abs(((pl.col("tctk") - pl.col("ghs")) / pl.col("tctk")))*100).alias("ghs"),
        (abs(((pl.col("tctk") - pl.col("meta")) / pl.col("tctk")))*100).alias("meta"),
    ).describe()

statistic,ghs,meta
str,f64,f64
"""count""",64.0,64.0
"""null_count""",0.0,0.0
"""mean""",12.177615,9.597109
"""std""",12.916889,9.512348
"""min""",0.512046,0.075875
"""25%""",4.407771,4.63957
"""50%""",10.433251,7.06713
"""75%""",16.286552,11.857699
"""max""",91.757263,59.533221


In [2]:
import folium

df = pl.read_parquet("./long_bien.parquet")

m = folium.Map(location=(21.022672, 105.831461), zoom_start=14)

places = df.unique(subset=["id"]).sample(n=200).to_dicts()

for place in places:
    folium.Marker(
        location=[float(place["lat"]), float(place["lon"])],
        tooltip=place["name"],
        icon=folium.Icon(color="red"),
    ).add_to(m)
m

In [3]:
df

id,lat,lon,name,categories,address
str,f64,f64,str,str,str
"""ChIJgRbT3pqpNTERuz9pwVYx51c""",21.008869,105.881819,"""Red River Camping""","""campground,lodging,park,point_…","""Cự Khối,2V5J+GP Long Biên, Han…"
"""ChIJu2ZjBXepNTERHafpXpbRtfg""",21.0092,105.878937,"""Bãi bồi vườn nhãn Sông Hồng""","""tourist_attraction,point_of_in…","""2V5H+MHM, Cầu Vĩnh Tuy, Long B…"
"""ChIJYVeoRVepNTERuREHmFod2t0""",21.015227,105.880996,"""Công ty Purastar""","""general_contractor,point_of_in…","""2V8J+39X, Long Biên"""
"""ChIJgRbT3pqpNTERuz9pwVYx51c""",21.008869,105.881819,"""Red River Camping""","""campground,lodging,park,point_…","""Cự Khối,2V5J+GP Long Biên, Han…"
"""ChIJq6qqzk2pNTERrZa5WWCKh5o""",21.008374,105.890319,"""Công Ty Cổ Phần Cơ Khí Và Vận …","""point_of_interest,establishmen…","""Xóm Bãi, Phường Thanh Trì, Quậ…"
…,…,…,…,…,…
"""ChIJqzEJ5YOpNTERzng27d8V-vI""",21.07628,105.902071,"""Máy phun sơn công Nghiệp""","""point_of_interest,establishmen…","""01 Phố Thanh Am, Thượng Thanh,…"
"""ChIJB1xSD1WpNTERBTxOEBxE_U0""",21.076255,105.902077,"""Áo đoàn thanh niên khoác gió t…","""clothing_store,store,point_of_…","""1 Phố Thanh Am, Thượng Thanh,3…"
"""ChIJtX8_UeepNTERJ5t9fSJPOu8""",21.076255,105.902077,"""Suli cafe""","""cafe,store,food,point_of_inter…","""1 Phố Thanh Am, Thượng Thanh,3…"
